### Reflectors and Projector

In [17]:
# Must run this. 
from numpy import sign, identity, array, vstack, zeros, triu, tril, sqrt, set_printoptions, arange, newaxis
from numpy.linalg import norm, cond
from numpy.random import rand
from math import isclose
from statistics import mean
import numpy as np
eye = identity
arr = array
set_printoptions(precision=4)

So, let's first of all, take a look at the Householder Reflector

In [18]:
v = arr([[1], [1]])

The $v$ vector will defines a perpendicular hyper plane which is going to be the reflector, in this case, the reflection plane is just the vector $(-1, 1)$, which lies in the reflection plane. 

In [19]:
P = (eye(2) - 2*(v@v.conj().T)/(v.conj().T@v))
print(P)

[[ 0. -1.]
 [-1.  0.]]


As we can see that the matrix indeed performs a reflection operations on vectors, and the plane of reflection is (-1, 1); by the way, it's a reflector but it's also a projector, it's just different interpretation on the same thing. 

In [20]:
def reflect(x, y):
    """
        projection matrix that project the x onto the plane y lines in.
            * It will return one of the hyperplane with more numerical accurancy, to get the other hyperplane
            the also project to y after the reflection, just take the negative value of it and then it will be down.
        :param x: 
            Column vector 
        :param y: 
            Column vector
    """
    if len(x.shape) == 2 and len(y.shape) == 2:
        y = (y/sqrt(norm(y)))*sqrt(norm(x))  # Y points in y direction and have length of x 
        v = -sign(x[0])*y - x                # Chose the plane to reflec, with better precision
        return (eye(y.shape[0]) - 2*(v@v.conj().T)/(v.conj().T@v))
    else:
        return None
    
def hyperplane_reflect(v):
    """
        Given the vector v, the orthogonal vector the hyperplane, this will produce a reflection that reflects 
        vector with this hyperplane 
        :para v:
            The v vector will have to be a column vector. 
    """
    VHeight = v.shape[0];
    P = (eye(VHeight) - 2*(v@v.conj().T)/(v.conj().T@v))
    return P
    

Now let's try this out with our vector on the householder reflector. 

In [35]:
P = reflect(arr([[1], [1]]), arr([[-1], [1]]))
print(P)
print(P@arr([[1], [1]]))

[[ 1.  0.]
 [ 0. -1.]]
[[ 1.]
 [-1.]]


Let's try out the one vector reflector. 



In [25]:
YAxisReflect = hyperplane_reflect(arr([[-1], [1]]))
print(YAxisReflect)


[[0. 1.]
 [1. 0.]]


### Designing the Householder Triangularization

Let's just make sure we can vstack matrices perperly. And it does the norm properly. 

In [26]:
stacked = vstack((arr([[1]]), arr([[2]])))
print(stacked)
zeros((3,3))
norm([2,0])

[[1]
 [2]]


2.0

Here, we need to take notes one the dimension of matrix slicing, when matrix it's sliced with regular slicer, the dimension doesn't always preserve. 



In [36]:
TheMatrix = arr([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(TheMatrix[:, 1])
print(TheMatrix[:, [1]])
print(TheMatrix[1, :])
print(TheMatrix[[1], :])

[2 5 8]
[[2]
 [5]
 [8]]
[4 5 6]
[[4 5 6]]


Let's also try the Sub matrix assignment 

In [37]:
TheMatrix[1:, 1:] = arr([[1, 1], [1, 1]])
print(TheMatrix)
TheMatrix[0:3, 0:3] = arr([[1, 1, 1]]*3) 
print(TheMatrix)

[[1 2 3]
 [4 1 1]
 [7 1 1]]
[[1 1 1]
 [1 1 1]
 [1 1 1]]


A warning about slicing stuff out of the matrix! Is it mutable slice? 


In [38]:
TheMatrix = arr([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
TheSubMatrix = TheMatrix[0:1, [0, 1]]
print(TheSubMatrix)
TheSubMatrix[0, 0] = -1
print(TheSubMatrix)
print(TheMatrix)

[[1 2]]
[[-1  2]]
[[1 2 3]
 [4 5 6]
 [7 8 9]]


Let's try some trikier example for this scheme, because there are pathological inputs that can destroy the algorithm, this is the case because the array tends to conserve the datatype. 

In [39]:
R = arr([[0, 1], [1, 0], [1, 0]])
print(R.dtype)
Temp =  arr([[-1.41421], [0], [0]])
R[0:, [0]] = Temp
print(R)

int32
[[-1  1]
 [ 0  0]
 [ 0  0]]


### Actual Implementation of Householder Triangularization

In [85]:
def qr_factor(R):
    """
        Performs a Householder Transformation on the given matrix A, 
        full QR Decomposition. 
    """
    R = R.copy().astype("float64")
    assert len(R.shape) == 2
    m, n = R.shape[0], R.shape[1]
    Q = eye(m)
    for K in range((n - 1) if n == m else n):
        z = R[K:m, [K]]
        v = zeros((z.shape[0], 1))
        NormZ = norm(z)
        if isclose(NormZ, 0): raise Exception("Rank Defecit")
        v[0, 0] = (1 if z[0] < 0 else -1)*NormZ
        v = v - z
        v = v/norm(v)
        J = list(range(n))
        R[K: m, :n] = R[K: m, J] - 2*(v@v.T)@R[K: m, J]
        J = list(range(m))
        Q[K: m, :m] = Q[K: m, J] - 2*(v@v.T)@Q[K: m, J]
    return Q.T, triu(R)


Let's try and run it on a simple 2 by 2 matrix. 

In [42]:
A = arr([[0, 1], [1, 0], [1, 0]])
print(A)
Q, R = qr_factor(A)
print("These are the factors: ")
print(Q)
print(R)
print("Matrix Reconstructed: ")
print(Q@R)

[[0 1]
 [1 0]
 [1 0]]
These are the factors: 
[[-2.2204e-16  1.0000e+00  1.1102e-16]
 [-7.0711e-01  5.5511e-17 -7.0711e-01]
 [-7.0711e-01 -5.5511e-17  7.0711e-01]]
[[-1.4142e+00 -2.2204e-16]
 [ 0.0000e+00  1.0000e+00]
 [ 0.0000e+00  0.0000e+00]]
Matrix Reconstructed: 
[[3.1402e-16 1.0000e+00]
 [1.0000e+00 2.1252e-16]
 [1.0000e+00 1.0150e-16]]


let's try and run it on any random 2 by 2 matrix

In [43]:
M = rand(2,2)*(10)
M = M.round(0)
print("This is the original matrix: ")
print(M)
Q, R = qr_factor(M)
print("This is its factor, Q, R")
print(Q)
print(R)
print("This is the reconstruction of the matrix: ")
print(Q@R)
print("This is Q Q transpose")
print(Q@Q.T)

This is the original matrix: 
[[6. 7.]
 [8. 8.]]
This is its factor, Q, R
[[-0.6 -0.8]
 [-0.8  0.6]]
[[-10.  -10.6]
 [  0.   -0.8]]
This is the reconstruction of the matrix: 
[[6. 7.]
 [8. 8.]]
This is Q Q transpose
[[ 1.0000e+00 -1.6653e-16]
 [-1.6653e-16  1.0000e+00]]


it seems to be working, let's try some bigger matrix, like 3 by 3 matrix with integers. 

In [44]:
M = rand(3, 3)*(10)
M[0, 0] = 0
M = M.round(0)
print("This is the original matrix: ")
print(M)
Q, R = qr_factor(M)
print("This is its factor, Q, R")
print(Q)
print(R)
print("This is the reconstruction of the matrix: ")
print(Q@R)
print("This is Q Q transpose")
print(Q@Q.T)

This is the original matrix: 
[[0. 9. 0.]
 [9. 7. 4.]
 [7. 8. 1.]]
This is its factor, Q, R
[[-2.2204e-16  9.7579e-01 -2.1871e-01]
 [-7.8935e-01 -1.3428e-01 -5.9908e-01]
 [-6.1394e-01  1.7264e-01  7.7024e-01]]
[[-11.4018 -10.437   -3.7713]
 [  0.       9.2233  -0.3645]
 [  0.       0.      -1.6261]]
This is the reconstruction of the matrix: 
[[2.5317e-15 9.0000e+00 1.3323e-15]
 [9.0000e+00 7.0000e+00 4.0000e+00]
 [7.0000e+00 8.0000e+00 1.0000e+00]]
This is Q Q transpose
[[1.0000e+00 2.7756e-16 5.5511e-17]
 [2.7756e-16 1.0000e+00 1.1102e-16]
 [5.5511e-17 1.1102e-16 1.0000e+00]]


In [77]:
M = rand(5, 4)*(10)
M = M.round(0)
print("The random matrix")
print(M)
Q, R = qr_factor(M)
print("This is the QR factors: ")
print(Q)
print(R)
print(norm((Q@R - M)))
print("Recovered Matrix: ")
print(Q@R)
print(norm(Q@Q.T - eye(Q.shape[0])))

The random matrix
[[3. 3. 1. 4.]
 [3. 3. 4. 3.]
 [2. 4. 8. 8.]
 [9. 8. 5. 0.]
 [8. 9. 5. 7.]]
This is the QR factors: 
[[-0.2321 -0.0221  0.2266  0.8189 -0.473 ]
 [-0.2321 -0.0221 -0.4833  0.4715  0.6998]
 [-0.1548  0.8055 -0.4766 -0.0686 -0.3089]
 [-0.6964 -0.4764 -0.3472 -0.2842 -0.2944]
 [-0.6191  0.3512  0.606  -0.147   0.3233]]
[[-12.9228 -13.155   -8.9763  -7.1966]
 [  0.       2.4385   5.7069   8.747 ]
 [  0.       0.      -4.2256  -0.1142]
 [  0.       0.       0.       3.1122]
 [  0.       0.       0.       0.    ]]
5.073123991470691e-15
Recovered Matrix: 
[[3.0000e+00 3.0000e+00 1.0000e+00 4.0000e+00]
 [3.0000e+00 3.0000e+00 4.0000e+00 3.0000e+00]
 [2.0000e+00 4.0000e+00 8.0000e+00 8.0000e+00]
 [9.0000e+00 8.0000e+00 5.0000e+00 6.6613e-16]
 [8.0000e+00 9.0000e+00 5.0000e+00 7.0000e+00]]
8.570587382375211e-16


#### Modifications so it Can Decompose all Matrices
For Rank Defecit matrix, the resulting R matrix will not be strictly diagonal anymore, so in that case, we will have accept that part, and in our original algorithm, we will make the choice of skipping zero columns while triangularzing.

The resulting $Q$ matrix will still be orthogonalt, that part is good.

In [30]:
def full_qr_factor(R):
    """
        Performs a Householder Transformation on the given matrix A, 
        full QR Decomposition. 
    """
    R = R.copy().astype("float64")
    assert len(R.shape) == 2
    m, n = R.shape[0], R.shape[1]
    Q = eye(m)
    for K in range((n - 1) if n == m else n):
        z = R[K:m, [K]]
        v = zeros((z.shape[0], 1))
        NormZ = norm(z)
        if isclose(NormZ, 0): continue
        v[0, 0] = (1 if z[0] < 0 else -1)*NormZ
        v = v - z
        v = v/norm(v)
        J = list(range(n))
        P = v@v.T
        R[K: m, :n] = R[K: m, J] - 2*P@R[K: m, J]
        J = list(range(m))
        Q[K: m, :m] = Q[K: m, J] - 2*P@Q[K: m, J]
    return Q.T, triu(R)

In [31]:
A = arr([[0, 1, 0], [1, 0, 1], [1, 0, 1]])
Q, R = full_qr_factor(A)
print("Q, R factor of the matrix A: ")
print(Q)
print(R)
print("Reconstructing the matrix: ")
print(Q@R)

Q, R factor of the matrix A: 
[[-2.2204e-16  1.0000e+00  1.1102e-16]
 [-7.0711e-01  5.5511e-17 -7.0711e-01]
 [-7.0711e-01 -5.5511e-17  7.0711e-01]]
[[-1.4142e+00 -2.2204e-16 -1.4142e+00]
 [ 0.0000e+00  1.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00  0.0000e+00]]
Reconstructing the matrix: 
[[3.1402e-16 1.0000e+00 3.1402e-16]
 [1.0000e+00 2.1252e-16 1.0000e+00]
 [1.0000e+00 1.0150e-16 1.0000e+00]]


### Efficiency and Accuracy of Householder Triangularization

In [36]:
def rand_skinny_matrix_QR_factor(m, n):
    TotalTrials = 100
    ReconstructionErrors = []
    for _ in range(TotalTrials):
        RandMatrix = rand(m, n)
        Q, R = qr_factor(RandMatrix)
        Error = norm(Q@R - RandMatrix)
        ReconstructionErrors.append(Error)
    return ReconstructionErrors 

In [84]:
print(f"The average element wise error of matrix reconstruction is: {mean(rand_skinny_matrix_QR_factor(100, 100))/1e4}")

The average element wise error of matrix reconstruction is: 5.1353992082016946e-18


Let's try it on some ill-conditioned matrices and see how it behaves. 

In [18]:
IllMatrix = arange(1, 100)[newaxis, :]
IllMatrix = 1/(IllMatrix.T@IllMatrix)
print(IllMatrix)
print(f"The condition number of the matrix is: {cond(IllMatrix)}")

[[1.0000e+00 5.0000e-01 3.3333e-01 ... 1.0309e-02 1.0204e-02 1.0101e-02]
 [5.0000e-01 2.5000e-01 1.6667e-01 ... 5.1546e-03 5.1020e-03 5.0505e-03]
 [3.3333e-01 1.6667e-01 1.1111e-01 ... 3.4364e-03 3.4014e-03 3.3670e-03]
 ...
 [1.0309e-02 5.1546e-03 3.4364e-03 ... 1.0628e-04 1.0520e-04 1.0413e-04]
 [1.0204e-02 5.1020e-03 3.4014e-03 ... 1.0520e-04 1.0412e-04 1.0307e-04]
 [1.0101e-02 5.0505e-03 3.3670e-03 ... 1.0413e-04 1.0307e-04 1.0203e-04]]
The condition number of the matrix is: 8.478940074588704e+35


In [35]:
Q, R = full_qr_factor(IllMatrix)
print(f"The Error for reconstructing the matrix is: {norm(Q@R - IllMatrix)/10000}")
print(f"The errors on orthogonality of the Q matrix: {norm(Q@Q.T - eye(Q.shape[0]))}")

The Error for reconstructing the matrix is: 1.1636453269059198e-19
The errors on orthogonality of the Q matrix: 1.2891577691211865e-14
